# Install packages

In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install xformers
!pip install -U sentence-transformers

In [ ]:
! pip install jsonlines
! pip install fast_ml --quiet
! pip install transformers
! pip install nltk
! python -m nltk.downloader all
! pip install unidecode

In [ ]:
from unidecode import unidecode
import nltk
from nltk import word_tokenize, WordNetLemmatizer
from nltk.corpus import stopwords
import string
import tensorflow as tf
import transformers
from textblob import TextBlob
import os
import json
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModel, AutoModelForSequenceClassification,AutoTokenizer,pipeline

# Data Loading and transformation

In [ ]:
# Load data
df_total = pd.read_parquet('/content/drive/MyDrive/Dissertation/Data/df.parquet')
# One-shot sample
df_label_unique_sample = df_total.groupby('label_cat', group_keys=False).apply(lambda df: df.sample(1))
# Create label mapping
label = list(df_label_unique_sample['label'])
id = list(df_label_unique_sample['label_cat'])
label_to_id = dict(zip(label,id))
id_to_label = dict(zip(id,label))

In [ ]:
# Benchmark sentence similarity data
binary_classification_data = pd.read_excel('/content/drive/MyDrive/Dissertation/Label_Classification JC comments.xlsx')
binary_classification_data['Label_text'] = binary_classification_data['Label'].map(id_to_label)

In [ ]:
# Split into Yes only lists and variables with Yes&No text
Label_yes = [20,0,21,25,4,8,10,5,23,19,22,28,27,30]
Label_yes_no = [3,6,7,9,18]
total_label = list(range(0,31))
total_second_stage_label  = Label_yes + Label_yes_no
total_first_stage_label  = [i for i in total_label if i not in total_second_stage_label]

# Define functions

In [ ]:
#Import sentence similarity model
model_infloat = SentenceTransformer('intfloat/e5-small-v2')

In [ ]:
def sent_similarity(model,label_cat, label,text,  target_sent,threshold):
  #text_extract['Label_cat'] = text_extract['Label'].map(label_to_id)
    # Text from the text extraction output
  # Label of this row of extraction
  label_extract = label_cat
  label_extract_text = label
  df = target_sent[target_sent['Label']==label_extract]
  # check whether the label belongs to Label_yes only or Label_yes_no
  if label_extract in Label_yes:
  # Target sentence from the Yes-only label
    target_sent_yes = list(df['Sentence (for Yes)'])
    text_score = sent_sim_func(model,text,target_sent_yes,label_extract_text)
    # Return binary classification over threshold
    score = text_score['Score']
    Label = text_score['Label']
    if score >=threshold:
        dic = {'Label':Label, 'Result': 'Yes'}
    else:
        dic = {'Label':Label, 'Result': 'No'}
  else: # Belongs to yes/no label
      yes_sent = list(df['Sentence (for Yes)'])
      no_sent = list(df['Sentence (for No)'])
      clean_no_sent = [x for x in no_sent if str(x) != 'nan']
      total_sent = yes_sent + clean_no_sent
      text_score = sent_sim_func(model,text,total_sent,label_extract_text)
      score = text_score['Score']
      Label = text_score['Label']
      if score >=threshold:
        # Check whther it is a Yes or No score
        if text_score['target_text'] in yes_sent:
          dic = {'Label':Label, 'Result': 'Yes'}
        else:
          dic = {'Label':Label, 'Result': 'No'}
      else:
        dic = {'Label':Label, 'Result': 'None'}
  return dic

In [ ]:
# Define a general function to find simialrity for labels that exists from the first classification model
def sent_sim_func(model,source_sent,target_sent, label):
  max_score = []
  for x in range(len(source_sent)):
    for y in range(len(target_sent)):
        embedding_1 = model.encode(source_sent[x], convert_to_tensor=True)
        embedding_2 = model.encode(target_sent[y], convert_to_tensor=True)
        score = float(util.pytorch_cos_sim(embedding_1, embedding_2)[0][0])
        target_text = target_sent[y]
        source_text = source_sent[x]
        dic = {'Text_extracted':source_text,'target_text':target_text,
               'Score':score, 'Label': label}
        max_score.append(dic)
        y+=1
    x+=1
  max_score_all = [d.get('Score') for d in max_score]
  max_score_index = max_score_all.index(max(max_score_all))
  max_score_final = max_score[max_score_index]
  return max_score_final